### Najpre ucitajmo dataset sa mog githaba (ne mora uvek da se ucitava lokalno sa uredjaja, moze i sa interneta)

In [10]:
import pandas as pd
brit = pd.read_csv('https://raw.githubusercontent.com/Niecmano/sempandas/main/data/brit_sredjen.csv')
print(brit.shape)
brit.sample(13)

(1076, 27)


,Title,AlbumName,Record Label,Performer,Track.number,Song.duration,Composer,Year,Genre,Styles,...,tempo,valence,duration_ms,popularity,shake.the.audience,romantic,obscene,family.spiritual,sadness,First composer
60,Look Before You Leap,Satisfied with You,BMG,The Dave Clark Five,5,2:18,Dave Clark / Lenny Davidson,1966,Pop/Rock,"British Invasion, Rock & Roll",...,135.250,0.6860,138893.0,5.0,0.002392,0.071605,0.002392,0.180122,0.362341,Dave Clark
981,Boris the Spider,A Quick One,MCA,The Who,2,2:26,John Entwistle,1966,Pop/Rock,"Album Rock, British Invasion, Contemporary Pop...",...,98.301,0.4410,149533.0,38.0,0.120855,0.162737,0.001595,0.013656,0.002632,John Entwistle
487,Without You,Ferry Cross the Mersey [Original Soundtrack],Japanese Import,Gerry & the Pacemakers,19,2:55,Gerry Marsden,1965,"Pop/Rock, Stage & Screen","AM Pop, British Invasion, Contemporary Pop/Roc...",...,94.015,0.2810,176187.0,3.0,0.000849,0.000849,0.000849,0.000849,0.380978,Gerry Marsden
974,Come Now,Cellophane,Repertoire,The Troggs,18,2:20,Ronnie Bond / Chris Britton / Tony Murray / Re...,1967,Pop/Rock,"British Invasion, AM Pop, Contemporary Pop/Rock",...,173.957,0.8560,139987.0,7.0,0.002632,0.000675,0.491090,0.000975,0.000658,Ronnie Bond
365,Lullaby,North Country Maid,Universal,Marianne Faithfull,11,2:40,Jon Mark,1966,Pop/Rock,"British Invasion, Folk-Rock, Baroque Pop",...,121.747,0.2060,159813.0,0.0,0.019388,0.000907,0.000907,0.076870,0.000907,Jon Mark
230,When Your Love Has Gone,Distant Shores,Sundazed,Chad & Jeremy,3,2:41,Bobby Goldsboro,1966,Pop/Rock,"Sunshine Pop, AM Pop, Baroque Pop, British Inv...",...,109.301,0.4980,155040.0,2.0,0.003096,0.003096,0.003096,0.003096,0.003096,Bobby Goldsboro
26,Say You Want Me,Coast to Coast,BMG,The Dave Clark Five,7,1:45,Dave Clark / Lenny Davidson,1965,Pop/Rock,"British Invasion, Early Pop/Rock, Rock & Roll",...,144.916,0.9060,105409.0,3.0,0.003509,0.132075,0.003509,0.003509,0.073726,Dave Clark
538,Gotta Get Away,Hold On!,Ausverkau,Herman's Hermits,8,1:55,Fred Karger / Ben Weisman,1966,"Pop/Rock, Stage & Screen","British Invasion, AM Pop",...,112.860,0.9450,113080.0,1.0,0.051075,0.087138,0.188904,0.000923,0.000923,Fred Karger
473,Fall in Love,Ferry Cross the Mersey [Original Soundtrack],Japanese Import,Gerry & the Pacemakers,3,1:49,Gerry Marsden,1965,"Pop/Rock, Stage & Screen","AM Pop, British Invasion, Contemporary Pop/Roc...",...,116.265,0.9040,109467.0,7.0,0.001462,0.001462,0.001462,0.001462,0.035564,Gerry Marsden
456,Whistle While You Work,In Disneyland,Rhino,Freddie & the Dreamers,9,2:05,Frank Churchill / Larry Morey,1966,Pop/Rock,"AM Pop, British Invasion",...,115.061,0.7310,125200.0,0.0,0.003096,0.003096,0.003096,0.112679,0.003096,Frank Churchill


### Dati pandas df cemo prebaciti u spark.. Pravimo semu datafrejma - StructType koji sadrzi StructField za svaku kolonu.

In [11]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, DateType

spark = SparkSession.builder.appName("Britanski bendovi").getOrCreate()
brit.rename(columns={'Song.duration':'duration'}, inplace=True)
schema = StructType([StructField(dataType=StringType(),name='Title',nullable=True),
                StructField(dataType=StringType(),name='Performer',nullable=True),
                StructField(dataType=DateType(),name='duration',nullable=True),
                StructField(dataType=StringType(),name='AlbumName',nullable=True),StructField(dataType=StringType(),name='Genre',nullable=True),StructField(dataType=FloatType(),name='energy',nullable=True)])
brit['Title']=brit['Title'].astype(str)
brit['AlbumName']=brit['AlbumName'].astype(str)
brit['Performer']=brit['Performer'].astype(str)
# brit.dtypes
bsp = spark.createDataFrame(brit[schema.fieldNames()])
print(type(bsp))
bsp.show(300)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+--------------------+--------+--------------------+--------------------+------+
|               Title|           Performer|duration|           AlbumName|               Genre|energy|
+--------------------+--------------------+--------+--------------------+--------------------+------+
|     Glad All Over  | The Dave Clark Five|    2:43|       Glad All Over|            Pop/Rock| 0.803|
|   All of the Time  | The Dave Clark Five|    2:15|       Glad All Over|            Pop/Rock| 0.667|
|   Crying Over You  | The Dave Clark Five|    2:09|       Glad All Over|            Pop/Rock|  0.36|
|              Stay  | The Dave Clark Five|    2:09|       Glad All Over|            Pop/Rock| 0.926|
|          Chaquita  | The Dave Clark Five|    2:13|       Glad All Over|            Pop/Rock| 0.891|
|     Do You Love Me | The Dave Clark Five|    2:43|       Glad All Over|            Pop/Rock| 0.913|
|   Bits and Pieces  | The Dave Clark Fi

In [4]:
bsp.select(['Title','duration']).show(10)

+--------------------+--------+
|               Title|duration|
+--------------------+--------+
|     Glad All Over  |    2:43|
|   All of the Time  |    2:15|
|   Crying Over You  |    2:09|
|              Stay  |    2:09|
|          Chaquita  |    2:13|
|     Do You Love Me |    2:43|
|   Bits and Pieces  |    1:58|
|        I Know You  |    2:00|
|              Time  |    2:19|
|Who Do You Think ...|    2:10|
+--------------------+--------+
only showing top 10 rows



In [5]:
bsp.groupby('AlbumName').agg({"energy": "SUM"}).show()

+--------------------+------------------+
|           AlbumName|       sum(energy)|
+--------------------+------------------+
|          5 X 5 [US]|5.4830000000000005|
|   Weekend in London| 7.395999999999998|
|  Satisfied with You|             6.331|
|       American Tour|             6.327|
|        Try Too Hard|             7.408|
|       Glad All Over|7.4190000000000005|
| I Like It Like That|             2.086|
|      Coast to Coast|             7.177|
|Magical Mystery Tour| 4.609999999999999|
|You Got What It T...|             6.696|
|    The Animals [US]| 7.351999999999999|
|           Animalism|             0.564|
|     Winds of Change| 5.646999999999999|
| The Animals on Tour|             7.727|
|       Animalization|             8.324|
|            Revolver|             7.428|
|  A Hard Day's Night|             7.924|
|               Help!|              8.77|
|    Beatles for Sale| 6.987000000000001|
|         Rubber Soul| 7.550999999999999|
+--------------------+------------

# Korisnicki def. fja i njena primena na kolonu df-a

In [6]:
from pyspark.sql.functions import udf, collect_set


def usek(traj):
    min,sek = map(int,traj.split(':'))
    return min*60+sek
usec = udf(usek, IntegerType())

novi = bsp.withColumn('trajanje',usec(bsp['duration']))
novi.show()

+--------------------+-------------------+--------+-------------+--------+------+--------+
|               Title|          Performer|duration|    AlbumName|   Genre|energy|trajanje|
+--------------------+-------------------+--------+-------------+--------+------+--------+
|     Glad All Over  |The Dave Clark Five|    2:43|Glad All Over|Pop/Rock| 0.803|     163|
|   All of the Time  |The Dave Clark Five|    2:15|Glad All Over|Pop/Rock| 0.667|     135|
|   Crying Over You  |The Dave Clark Five|    2:09|Glad All Over|Pop/Rock|  0.36|     129|
|              Stay  |The Dave Clark Five|    2:09|Glad All Over|Pop/Rock| 0.926|     129|
|          Chaquita  |The Dave Clark Five|    2:13|Glad All Over|Pop/Rock| 0.891|     133|
|     Do You Love Me |The Dave Clark Five|    2:43|Glad All Over|Pop/Rock| 0.913|     163|
|   Bits and Pieces  |The Dave Clark Five|    1:58|Glad All Over|Pop/Rock| 0.835|     118|
|        I Know You  |The Dave Clark Five|    2:00|Glad All Over|Pop/Rock| 0.872|     120|

# Pravljenje pogleda i sql upiti

In [7]:
spark.sql("DROP VIEW IF EXISTS brits")
novi.createTempView('brits')
spark.sql('SELECT * from brits where energy<0.6 and trajanje>150').show()

+--------------------+--------------------+--------+-------------------+---------------+------+--------+
|               Title|           Performer|duration|          AlbumName|          Genre|energy|trajanje|
+--------------------+--------------------+--------+-------------------+---------------+------+--------+
|Whenever You're A...| The Dave Clark Five|    2:54|      American Tour|       Pop/Rock| 0.493|     174|
|       Blue Monday  | The Dave Clark Five|    2:56|      American Tour|       Pop/Rock|  0.52|     176|
|         Come Home  | The Dave Clark Five|    2:48|  Weekend in London|       Pop/Rock| 0.361|     168|
|    Hurting Inside  | The Dave Clark Five|    2:37|  Weekend in London|       Pop/Rock| 0.354|     157|
|I'll Be Yours My ...| The Dave Clark Five|    2:42|I Like It Like That|       Pop/Rock| 0.524|     162|
| Sitting Here Baby  | The Dave Clark Five|    3:00|         5 X 5 [US]|       Pop/Rock|  0.43|     180|
|You Don't Want My...| The Dave Clark Five|    2:48|   

# Pravljenje datafrejma od nule, join

In [12]:
brs = bsp.withColumnRenamed("Title","Song title")
brs.sort('duration',ascending=False).show(50)

+--------------------+--------------------+--------+--------------------+--------------------+------+
|          Song title|           Performer|duration|           AlbumName|               Genre|energy|
+--------------------+--------------------+--------+--------------------+--------------------+------+
|A Quick One While...|             The Who|    9:05|         A Quick One|            Pop/Rock| 0.602|
|Sing This All Tog...|  The Rolling Stones|    8:33|Their Satanic Maj...|            Pop/Rock| 0.818|
|Lunch Break: Peak...|The Moody Blues f...|    7:41|Days of Future Pa...|            Pop/Rock| 0.444|
|  Talkin' 'Bout You |         The Animals|    7:03|    The Animals [US]|            Pop/Rock| 0.878|
|Legend of a Girl ...|             Donovan|    6:50|Sunshine Superman...|      Pop/Rock, Folk| 0.238|
|      Rest in Peace |Chad & Jeremy fea...|    6:47|Of Cabbages and K...|            Pop/Rock|  0.34|
|    Paint It Black  |Eric Burdon & the...|    6:20|     Winds of Change|         

In [14]:
lst = [(40050,'St.Polten',25.4),(20033,'Innsbruck',22.9),(14005,'Drasenhofen',28.4),(21030,'Villach',23.3),(90800,'Vaduz',21.9)]
sch = ['id','stadt','temperatur']
wea = spark.createDataFrame(lst,sch)
wea.show()

+-----+-----------+----------+
|   id|      stadt|temperatur|
+-----+-----------+----------+
|40050|  St.Polten|      25.4|
|20033|  Innsbruck|      22.9|
|14005|Drasenhofen|      28.4|
|21030|    Villach|      23.3|
|90800|      Vaduz|      21.9|
+-----+-----------+----------+



In [15]:
l2 = [(20033,False),(14005,True)]
w2 = spark.createDataFrame(l2,['id','regnet'])
w12 = wea.join(w2,on='id')
w12.show()

+-----+-----------+----------+------+
|   id|      stadt|temperatur|regnet|
+-----+-----------+----------+------+
|14005|Drasenhofen|      28.4|  true|
|20033|  Innsbruck|      22.9| false|
+-----+-----------+----------+------+

